# Learn, search and save models (backend)

In [1]:
import pandas as pd

from models.helpers_talent import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, mean_squared_error
from models.VAE import *
from sklearn.manifold import TSNE
import os
import csv
plt.rcParams['figure.figsize'] = (15, 12)
save_dir = 'saved_models/tfp'

from keras.backend import manual_variable_initialization
manual_variable_initialization(True)

Using TensorFlow backend.


### Load data

In [2]:
# <editor-fold desc="DATA">
# ___ DATA
group = 1           # age class 'AK'
test_size = 0.2     # train-test split
sub = False         # subjective meassures

dat0 = pd.read_csv('data/dat_vs1', sep=' ', index_col=False)
dat = dat0[dat0['AK'] == group]
dat = dat.drop('AK', axis=1).reset_index(drop=True)
if sub:
    c = 16
else:
    c = 14
X_0 = dat.iloc[:, 2:c]
y_0 = dat.iloc[:, 16]
X, X_test, y, y_test = train_test_split(X_0, y_0, test_size=test_size, random_state=42, stratify=y_0)
X_names = list(X_0.columns)

X.loc[X['Gewicht']>70,:]

# ___ SCALING
# MinMax [0,1]
scaler = StandardScaler().fit(X)
df = scaler.transform(X)
# scaler_test = StandardScaler().fit(X_test)
df_test = scaler.transform(X_test)
y = np.asarray(y)
# </editor-fold>

## Search loop

####### SEARCH

pretrain_epochs = 5000
batch_size = 256
inter1_dim = 8
inter2_dim = 0
callback = EarlyStopping(monitor="loss", min_delta=0.0001, patience=20)
learning_rate = 0.01

result_dict = []
for latent_dim in range(3, 8):
    for beta in np.arange(0.4, 2, 0.1):
        vae, en, de, mod = learn(df, inter1_dim, inter2_dim, latent_dim, beta,
                                           learning_rate, result_dict, callback, pretrain_epochs, batch_size)

        mod_name = mod
        path = save_dir + mod_name

        if not os.path.exists(path):
            os.mkdir(path)

        vae.save_weights(path + '/weights.h5')
        vae.save(path + '/vae.h5')
        en.save(path +'/en.h5')
        de.save(path +'/de.h5')

# save results as csv
import csv
loss_dic_names = ['name', 'inter1_dim', 'inter2_dim', 'latent_dim', 'beta', 'learning_rate', 'elbo', 'mean_loglik', 'mean_KL']
with open(save_dir+'/model_results_search.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=loss_dic_names)
    writer.writeheader()
    writer.writerows(result_dict)

## Beta - Loss dict for plots

pretrain_epochs = 500
batch_size = 256
inter1_dim = 8
inter2_dim = 0
latent_dim = 4
callback = EarlyStopping(monitor="loss", min_delta=0.0001, patience=20)
learning_rate = 0.01

loss_dict = []
for beta in np.arange(0.1, 2, 0.1):
    vae, en, de, mod = learn(df, inter1_dim, inter2_dim, latent_dim, beta,
                                 learning_rate, [], callback, pretrain_epochs, batch_size)
    temp = {'latent_dim':latent_dim, 'beta':beta, '-loglike': -mean_loglik(vae, df), 'KL':mean_KL(en, df, latent_dim), 'elbo':vae.evaluate(df,df)}
    loss_dict.append(temp)

### Save as csv

names = ['latent_dim', 'beta', '-loglike', 'KL', 'elbo']
with open(save_dir+'/beta_loss.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=names)
    writer.writeheader()
    writer.writerows(loss_dict)

## Learn specific model

In [7]:
##### LEARN #####

#model_results = pd.read_csv(save_dir + '/model_results.csv', index_col=False)

pretrain_epochs = 500
batch_size = 256
inter1_dim = 8
inter2_dim = 0
latent_dim = 5
learning_rate = 0.01
callback = EarlyStopping(monitor="loss", min_delta=0.0001, patience=20)
beta = 0.6
original_dim = df.shape[1]


result_dict = []
vae, encoder, decoder, mod = learn(df, inter1_dim, inter2_dim, latent_dim,
                                   beta, learning_rate, result_dict,
                                   callback, pretrain_epochs, batch_size)

Train on 5217 samples
Epoch 1/500
5217/5217 [==============================] - 3s 543us/sample - loss: 22.9606
Epoch 2/500
5217/5217 [==============================] - 0s 19us/sample - loss: 18.6035
Epoch 3/500
5217/5217 [==============================] - 0s 18us/sample - loss: 18.0071
Epoch 4/500
5217/5217 [==============================] - 0s 24us/sample - loss: 17.7027
Epoch 5/500
5217/5217 [==============================] - 0s 21us/sample - loss: 17.5044
Epoch 6/500
5217/5217 [==============================] - 0s 26us/sample - loss: 17.3934
Epoch 7/500
5217/5217 [==============================] - 0s 24us/sample - loss: 17.2966
Epoch 8/500
5217/5217 [==============================] - 0s 19us/sample - loss: 17.2070
Epoch 9/500
5217/5217 [==============================] - 0s 18us/sample - loss: 17.1528
Epoch 10/500
5217/5217 [==============================] - 0s 21us/sample - loss: 17.0703
Epoch 11/500
5217/5217 [==============================] - 0s 21us/sample - loss: 17.0096
Epoch 1

In [8]:
ldist = encoder(df)
assert isinstance(ldist, tfd.Distribution)
outdist = vae(df)
assert isinstance(outdist, tfd.Distribution)
meandist = decoder(ldist.mean())
assert isinstance(meandist, tfd.Distribution)


#plot_label_clusters(ldist.mean(), y, latent_dim)

In [9]:
prior = tfd.Independent(tfd.Normal(loc=0, scale=1.))
kl = []
for i in range(latent_dim):
    dist_temp = tfd.Independent(tfd.Normal(loc=np.asarray(ldist.mean()[:,i]).mean(), scale=np.asarray(ldist.stddev()[:,i]).mean()))
    kl.append(np.asarray(tfp.distributions.kl_divergence(prior, dist_temp)))
    print(i,kl[i])

0 1.0465181
1 0.0012437222
2 0.24836951
3 2.8911164
4 0.0019170791


## Save

In [10]:
###### SAVE ######
#mod_name = mod
mod_name = 'mvn_test'
path = save_dir + mod_name

if not os.path.exists(path):
    os.mkdir(path)

vae.save_weights(path + '/weights.h5')
vae.save(path + '/vae.h5')
encoder.save(path +'/en.h5')
decoder.save(path +'/de.h5')

In [224]:
# save results as csv
loss_dic_names = ['name', 'inter1_dim', 'inter2_dim', 'latent_dim', 'beta', 'learning_rate', 'elbo', 'mean_loglik', 'mean_KL']
with open(save_dir+'/model_results.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=loss_dic_names)
    writer.writeheader()
    writer.writerows(result_dict)

## Test

In [ ]:
###### LOAD ######
'''
model_results = pd.read_csv(save_dir + '/model_results.csv', index_col=False)
learning_rate = 0.01
vae, encoder, decoder = load_vae(save_dir, learning_rate)
latent_dim = encoder.output.event_shape[0]

ldist = encoder(df)
assert isinstance(ldist, tfd.Distribution)
outdist = vae(df)
assert isinstance(outdist, tfd.Distribution)
meandist = decoder(ldist.mean())
assert isinstance(meandist, tfd.Distribution)


plot_label_clusters(ldist.mean(), y, latent_dim)
'''